## Cross Base Evaluation
This notebook aims to examine the data status across all databases before initiating the data cleaning process. To understand which feature classes contain data or values within each database.

#### import necessary packages

In [5]:
import os
import csv
import arcpy
import pandas as pd

### Check the feature class in all GDBs

In this step, the goal is to output the data status in each database:
1. Feature Class: Identifying the feature class present in each database.
2. Type: Specifying the type of the feature class.
3. Feature Dataset: Easier identification of the location where the feature class is stored.
4. Presented: Indicating if the geodatabase (GDB) has this feature class and has a value ("1"), if the GDB has the feature class but no value ("0"), and if the GDB doesn't have this feature class, the cell is left blank.
5. Feature Count: Showing the number of data entries in each feature class.
6. Percentage of Non-NULL Fields, Number of Non-NULL Values, Number of NULL Values, Number of Total Values, Number of Fields: These five columns provide insights into the distribution of values and non-values within each feature class.


In [42]:
def checkGDB(gdb_path, output_csv_path):
    arcpy.env.workspace = gdb_path
    
    # Extract the last part of the GDB path
    gdb_basename = os.path.basename(gdb_path)

    # Create a CSV file for output
    with open(output_csv_path, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        
        # Write GDB name row
        csv_writer.writerow(["", "", "", f" {gdb_basename} ", "", "", "", ""])

        # Write header to CSV
        csv_writer.writerow(["Feature Class", "Type", "Feature Dataset", "Presented", "Feature Count",
                             "Percentage of Non-NULL Fields","Number of Non-NULL Values", "Number of NULL Values",
                             "Number of Total Values", "Number of fields"])
        
        # Check if Feature Datasets exist
        feature_datasets = arcpy.ListDatasets("", "")
        if feature_datasets:
            # Loop through FDs
            print(f"--- GDB {gdb_basename} ---")  # Print GDB name once
            for fd in feature_datasets:
                arcpy.env.workspace = os.path.join(gdb_path, fd)
                feature_classes = arcpy.ListFeatureClasses()

                print(f"--- {fd} ---")  # Print FD name

                for fc in feature_classes:
                    describe_and_calculate(fc, csv_writer, fd)
        else:
            # No Feature Datasets, directly process Feature Classes
            feature_classes = arcpy.ListFeatureClasses()
            for fc in feature_classes:
                describe_and_calculate(fc, csv_writer, "")

In [43]:
def describe_and_calculate(fc, csv_writer, fd):
    description = arcpy.Describe(fc)
    count_result = arcpy.management.GetCount(fc)
    feature_count = int(count_result.getOutput(0))
    
    if feature_count == 0:
        feature_status = "0"
    else:
        feature_status = "1"

    total_fields = 0
    non_null_fields = 0
    null_fields = 0
    percentage_with_values = ""

    # Obtain the fields with value
    fields = arcpy.ListFields(fc)

    for field in fields:
        with arcpy.da.SearchCursor(fc, [field.name]) as cursor:
            for row in cursor:
                for value in row:
                    total_fields += 1
                    if value is not None:
                        non_null_fields += 1
                    else:
                        null_fields += 1
    
    # Calculate the percentage of each attribute
    if total_fields > 0:
        percentage_with_values = (non_null_fields / total_fields) * 100

    # Write results to CSV
    csv_writer.writerow([fc, description.shapeType, fd, feature_status, feature_count, percentage_with_values, 
                         non_null_fields, null_fields, non_null_fields + null_fields, len(fields)])

In [44]:
list_of_fd_paths = [
    #r"D:\spring2024\AFRICOM\DoD SAFE-Z6ZsX3Em6bkXVeBq\CIP_Africa_Data_2023\Agadez_SDSFIE_MASTER_2022April\Agadez_Master_2022April\CIP_311_AB201.gdb",
    #r"D:\spring2024\AFRICOM\DoD SAFE-Z6ZsX3Em6bkXVeBq\CIP_Africa_Data_2023\Chabelley_SDSFIE_MASTER_March2023\Chebelley_40_MASTER.gdb",
    #r"D:\spring2024\AFRICOM\DoD SAFE-Z6ZsX3Em6bkXVeBq\CIP_Africa_Data_2023\Chabelley_SDSFIE_MASTER_March2023\Chabelley_SDSFIE_MASTER_March2023.gdb",
    #r"D:\spring2024\AFRICOM\DoD SAFE-Z6ZsX3Em6bkXVeBq\CIP_Africa_Data_2023\Chabelley_SDSFIE_MASTER_March2023\Chebelly_40_MASTER_CIP.gdb",
    #r"D:\spring2024\AFRICOM\DoD SAFE-Z6ZsX3Em6bkXVeBq\CIP_Africa_Data_2023\Niamey_SDSFIE_311_Master_9Jan2023\Niamey_SDSFIE_311_Master_9Jan2023.gdb",
    #r"D:\spring2024\AFRICOM\DoD SAFE-Z6ZsX3Em6bkXVeBq\CIP_Africa_Data_2023\SDSFIE_MandaBay_MASTER_2023\SDSFIE_05Apr2023.gdb"
    # updated gdb
    r'C:\Users\ma000551\Desktop\AFRICOM\AFRICOM_CLEAN_DATA\04_update\AB201_JAN2024.gdb',
    r'C:\Users\ma000551\Desktop\AFRICOM\AFRICOM_CLEAN_DATA\04_update\Niamey_SDSFIE_311_Master_18MAY2023.gdb'
]

In [45]:
# Define an empty DataFrame before the loop

for gdb_path in list_of_fd_paths:
    # Extract GDB filename without extension
    gdb_name = os.path.splitext(os.path.basename(gdb_path))[0]
    
    # Construct CSV filename using GDB name
    #output_csv_path = rf"D:\spring2024\cross_base_evaluation\{gdb_name}.csv"
    output_csv_path = rf"C:\Users\ma000551\Desktop\AFRICOM\AFRICOM_CLEAN_DATA\04_update\{gdb_name}.csv"
    
    checkGDB(gdb_path, output_csv_path)

#### merge by Feature Classes
merge all the GDB's evaluation CSVs together with same feature class, so that we can have the insight of cross base evaluation

In [9]:
list_of_csv_paths = [
    #r"D:\spring2024\cross_base_evaluation\Chebelley_40_MASTER.csv",
    #r"D:\spring2024\cross_base_evaluation\Chebelly_40_MASTER_CIP.csv",
    #r"D:\spring2024\cross_base_evaluation\Chabelley_SDSFIE_MASTER_March2023.csv",
    #r"D:\spring2024\cross_base_evaluation\CIP_311_AB201.csv",
    #r"D:\spring2024\cross_base_evaluation\Niamey_SDSFIE_311_Master_9Jan2023.csv",
    #r"D:\spring2024\cross_base_evaluation\SDSFIE_05Apr2023.csv"
    # updated csv
    r'C:\Users\ma000551\Desktop\AFRICOM\AFRICOM_CLEAN_DATA\04_update\AB201_JAN2024.csv',
    r'C:\Users\ma000551\Desktop\AFRICOM\AFRICOM_CLEAN_DATA\04_update\Niamey_SDSFIE_311_Master_18MAY2023.csv',
]


# Create an initial DataFrame with the Feature Class columns
merged_df = pd.DataFrame(columns=["Feature Class"].str.lower())

for csv_path in list_of_csv_paths:
    gdb_basename = os.path.basename(csv_path)

    # Read CSV file, skip the first row, and use the second row as header
    current_df = pd.read_csv(csv_path, skiprows=1)

    # Select only the desired columns
    current_df = current_df[["Feature Class", "Presented", "Feature Count"]]

    print(current_df)
    # Rename columns with suffixes
    current_df.columns = ["Feature Class", f"Presented_{gdb_basename}", f"Feature Count_{gdb_basename}"]


    # Merge DataFrames with custom suffixes
    merged_df = pd.merge(merged_df, current_df, on=['Feature Class'], how='outer')

# Specify the output path for the merged CSV file
#output_merged_csv_path = r"D:\spring2024\cross_base_evaluation\cross_base_evaluation_all.csv"
output_merged_csv_path = r"C:\Users\ma000551\Desktop\AFRICOM\AFRICOM_CLEAN_DATA\04_update\cross_base_evaluation_all_updated.csv"


# Save the merged DataFrame to a CSV file
merged_df.to_csv(output_merged_csv_path, index=False)


AttributeError: 'list' object has no attribute 'str'

In [10]:
list_of_csv_paths = [
    r'C:\Users\ma000551\Desktop\AFRICOM\AFRICOM_CLEAN_DATA\04_update\AB201_JAN2024.csv',
    r'C:\Users\ma000551\Desktop\AFRICOM\AFRICOM_CLEAN_DATA\04_update\Niamey_SDSFIE_311_Master_18MAY2023.csv',
]

# Create an initial DataFrame with the Feature Class columns
merged_df = pd.DataFrame(columns=["Feature Class"])

for csv_path in list_of_csv_paths:
    gdb_basename = os.path.basename(csv_path)

    # Read CSV file, skip the first row, and use the second row as header
    current_df = pd.read_csv(csv_path, skiprows=1)

    # Select only the desired columns
    current_df = current_df[["Feature Class", "Presented", "Feature Count"]]

    # Rename columns with suffixes
    current_df.columns = ["Feature Class", f"Presented_{gdb_basename}", f"Feature Count_{gdb_basename}"]

    # Convert 'Feature Class' values to lowercase
    current_df['Feature Class'] = current_df['Feature Class'].str.lower()

    # Merge DataFrames with custom suffixes
    merged_df = pd.merge(merged_df, current_df, on='Feature Class', how='outer')

# Specify the output path for the merged CSV file
output_merged_csv_path = r"C:\Users\ma000551\Desktop\AFRICOM\AFRICOM_CLEAN_DATA\04_update\cross_base_evaluation_all_updated.csv"

# Save the merged DataFrame to a CSV file
merged_df.to_csv(output_merged_csv_path, index=False)


In [8]:
# Read the first DataFrame from the updated CSV file
df1 = pd.read_csv(r"C:\Users\ma000551\Desktop\AFRICOM\AFRICOM_CLEAN_DATA\04_update\cross_base_evaluation_all_updated.csv")

# Read the second DataFrame from the original CSV file
df2 = pd.read_csv(r"C:\Users\ma000551\Desktop\AFRICOM\AFRICOM_CLEAN_DATA\04_update\cross_base_evaluation_all - cross_base_evaluation_all.csv")

# Convert 'Feature Class' values to lowercase in both DataFrames
df1['Feature Class'] = df1['Feature Class'].str.lower()
df2['Feature Class'] = df2['Feature Class'].str.lower()

# Merge the two DataFrames on the lowercase version of the 'Feature Class' column using an outer join
df3 = pd.merge(df1, df2, on='Feature Class')

# Write the merged DataFrame to a new CSV file
df3.to_csv(r"C:\Users\ma000551\Desktop\AFRICOM\AFRICOM_CLEAN_DATA\04_update\updated_cross_base_evaluation.csv", index=False)


Following this initial assessment, we can delve deeper into the data by utilizing CSV files. This will provide insights into the quantity of data with values, guiding our approach for continued data cleaning